# BMI565: Bioinformatics Programming & Scripting

#### (C) Michael Mooney (mooneymi@ohsu.edu)

## Week 11: Benchmarking and Optimizing Python Code

1. Benchmarking / Profiling in Python
2. Optimizing Code with Cython
3. Parallel Processing
    - `Multiprocessing` module
    - `pp` (Parallel Python) module
4. Final Exam Review

#### Requirements

- Python 2.7 or 3.5+
- `time`, `timeit`, and `profile` modules
- `scipy` and `numpy` modules
- `multiprocessing` module
- Parallel Python, `pp`, module
- Parallel Python for Python 2.7 can be installed with `pip install pp`.
- A Python 3 version of Parallel Python is here: [http://www.parallelpython.com/content/view/18/32/](http://www.parallelpython.com/content/view/18/32/)
- You can install from the .zip file: `pip3 install ~/Downloads/pp-1.6.4.4.zip`

In [ ]:
from __future__ import print_function, division
import numpy as np

## Benchmarking / Profiling

There are a number of ways to evaluate the performance of your Python code. Three useful modules are:

- `time`
- `timeit`
- `profile`

In [ ]:
## Define a function that determines if a number is prime
def isprime(n):
    """
    Returns the number if it is prime, otherwise returns None.
    """
    assert n > 0, "Number must be greater than 0!"
    if n < 2: return None
    for i in range(2,n):
        if n % i == 0:
            return None
    return n

def get_primes(min, max):
    result = []
    possible_primes = range(min,max+1)
    for n in possible_primes:
        result.append(isprime(n))

    prime_nums = [n for n in result if n is not None]
    return prime_nums

In [ ]:
## Binary search function
def bsearch(l, n):
    s = 0
    e = len(l) - 1
    while True:
        if s > e:
            return None
        mid = int((s + e)/2)
        if l[mid] < n:
            s = mid  + 1
        elif l[mid] > n:
            e = mid  - 1
        else:
            return mid

In [ ]:
## Recursive binary search function
def rec_bsearch(l,n,s=0,e=None):
    if e is None: e = len(l) - 1
    if s > e:
        return None
    mid = int((s + e)/2)
    if n == l[mid]:
        return mid
    elif n < l[mid]:
        return rec_bsearch(l,n,s,mid-1)
    else:
        return rec_bsearch(l,n,mid+1,e)

### `time` module

In [3]:
import time

def search_time(fun, N, M):
    runtimes = []
    nums = np.arange(M)
    start_time = time.time()
    for i in range(N):
        t0 = time.time()
        cmd = fun + "(nums, 3450)"
        idx = eval(cmd)
        runtimes.append(time.time() - t0)
    
    print("Total runtime: ", time.time() - start_time)
    print("Mean runtime: ", sum(runtimes)/len(runtimes))
    return None

In [ ]:
print("Binary Search:")
search_time("bsearch", 5000, 1000000)

### `timeit` module

In [ ]:
import timeit

## Get the runtime of a Python statement
timeit.timeit("bsearch(nums, 3450)", setup="from __main__ import bsearch; import numpy as np; nums = np.arange(1000000)", number=5000)

In [ ]:
## Create a timer and run it multiple times
timer = timeit.Timer("bsearch(nums, 3450)", setup="from __main__ import bsearch; import numpy as np; nums = np.arange(1000000)")
timer.repeat(3, number=5000)

### `profile` module

In [ ]:
import profile
nums = np.arange(100000000)
profile.run("rec_bsearch(nums, 3450)")

## Cython

Cython is a programming language that allows you to create C extensions for your Python programs, thereby improving performance. I'll show one simple example of how you might do this below, but for more details consult the Cython documentation (link in references). Before running the example code blocks below, you'll need to create two files and run a command to compile the Cython module. Note that the code shown below for the Cython implementation of the binary search algorithm is nearly identical to pure Python, with just a few minor tweaks (mostly accounting for data type conversions between Python/Numpy and Cython). 

Create a file called **'cy_bsearch.pyx'** that contains the following:

    #cython: boundscheck=False
    #cython: wraparound=False
    cimport numpy as np
    
    cpdef bsearch(np.ndarray[np.int64_t, ndim=1] L, int target, int start=0, end=None):
        """
        Binary search
        """
        cdef Py_ssize_t idx, mid
        
        if end is None: 
            end = len(L)-1
        while L.size > 0:
            idx = start + int((end - start)/2)
            mid = int((len(L)-1)/2)
            if L[mid] == target:
                return idx
            else:
                if target < L[mid]:
                    L = L[:mid]
                    end = idx - 1
                else:
                    L = L[mid+1:]
                    start = idx + 1
        return None


Create another file called **'setup.py'** that contains the following:

    from distutils.core import setup
    from distutils.extension import Extension
    from Cython.Distutils import build_ext
    
    import numpy
    
    ext = Extension("cy_bsearch", ["cy_bsearch.pyx"],
                    include_dirs = [numpy.get_include()])
    
    setup(ext_modules=[ext],
          cmdclass = {'build_ext': build_ext})

Finally, run the following at the command-line (this may produce a number of warnings, which can be ignored):

    python setup.py build_ext --inplace

This example is purely meant to illustrate how you might use Cython. However, because binary search is such a simple algorithm, we don't see any improvement in performance. This is an important point when it comes to optimization: **Over-optimization can be a real waste of time.** Rather than optimizing every last piece of code, you should focus only on those places where you will see real benefit.

In [1]:
## pyximport can be used to re-compile after changes,
## without having to run setup.py, but it's not necessary here
#import pyximport
#pyximport.install(setup_args={'include_dirs':[np.get_include()]})
from cy_bsearch import bsearch as cython_bsearch

In [11]:
import numpy as np
nums = np.arange(100000000, dtype='int64')
cython_bsearch(nums, 3450)

3450

In [4]:
import numpy as np
import time

def search_time(fun, N, M):
    runtimes = []
    nums = np.arange(M)
    start_time = time.time()
    for i in range(N):
        t0 = time.time()
        cmd = fun + "(nums, 3450)"
        idx = eval(cmd)
        runtimes.append(time.time() - t0)
    
    print("Total runtime: ", time.time() - start_time)
    print("Mean runtime: ", sum(runtimes)/len(runtimes))
    return None

In [5]:
print("Cython Binary Search:")
search_time("cython_bsearch", 5000, 1000000)

Cython Binary Search:
Total runtime:  0.1681051254272461
Mean runtime:  3.333468437194824e-05


In [10]:
nums.dtype

dtype('int64')

In [5]:
timeit.timeit("cython_bsearch(nums, 3450)", setup="from cy_bsearch import bsearch as cython_bsearch; import numpy as np; nums = np.arange(1000000)", number=5000)

NameError: name 'timeit' is not defined

## Parallel Processing

Parallel processing is a technique for improving the performance of a computational task, based on the idea that large problems can often be split into multiple smaller problems. These smaller problems can then be solved simultaneously (in parallel). Given the constraints of processor design and development, parallel computing (multi-processor machines) is now a common way to improve computational power.

There are numerous Python modules that allow you to take advantage of the computational power of multiple processors (the list below is not complete):

[https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)

Keep in mind that there is always some amount of overhead cost due to splitting a large task into multiple smaller task and then gathering/compiling the individual results. The efficiency gains achieved with parallel processing will depend on the individual tasks being performed and the amount of communication (data transfer) required.

In [1]:
import multiprocessing as mp
import pp

In [ ]:
## Find prime numbers serially
min_prime = 30000
max_prime = 50000

t0 = time.time()
prime_nums = get_primes(min_prime, max_prime)
t1 = time.time()

print("There are %d prime numbers between %d and %d." % (len(prime_nums), min_prime, max_prime))
print("Elapsed time:", t1 - t0)

In [ ]:
profile.run("get_primes(min_prime, max_prime)")

### `Multiprocessing` module

`Multiprocessing` is a module in Python's standard library that allows you to spawn multiple Python processes. It is an easy way to take advantage of multiple cores on a single machine.

[https://docs.python.org/2/library/multiprocessing.html](https://docs.python.org/2/library/multiprocessing.html)

In [2]:
## Get number of CPUs
mp.cpu_count()

8

In [6]:
## Find prime numbers using parallel processes
import time
from primes import isprime

min_prime = 30000
max_prime = 50000
possible_primes = range(min_prime,max_prime+1)

t2 = time.time()
pool = mp.Pool(processes=4)
result2 = pool.map(isprime, possible_primes)
prime_nums2 = [n for n in result2 if n is not None]
t3 = time.time()

## Make sure to close the processes created by Pool
pool.close()

print("There are %d prime numbers between %d and %d." % (len(prime_nums2), min_prime, max_prime))
print("Elapsed time:", t3 - t2)

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 2.287987232208252


In [ ]:
isprime(11)

### `pp` (Parallel Python) module

The Parallel Python module can be used to parallelize across multiple processors on a single machine, and also across multiple nodes of a computing cluster.

[http://www.parallelpython.com/](http://www.parallelpython.com/)

A beta version (1.6.4.4) that is ported to Python 3 can be found here:

[https://www.parallelpython.com/content/view/18/32/](https://www.parallelpython.com/content/view/18/32/)

In [ ]:
## Create pp job server
job_server = pp.Server(ncpus=4)
jobs = []

t4 = time.time()
## Submit jobs to pp server
for i in possible_primes:
    jobs.append(job_server.submit(isprime, (i,)))
## Wait for all jobs to finish
job_server.wait()
prime_nums3 = [job() for job in jobs if job() is not None]
t5 = time.time()

## Close the processes created by pp
job_server.destroy()

## Print results
print("There are %d prime numbers between %d and %d." % (len(prime_nums3), min_prime, max_prime))
print("Elapsed time:", t5 - t4)

## References

- [https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)
- [http://docs.cython.org/en/latest/](http://docs.cython.org/en/latest/)
- [https://docs.python.org/3.8/library/time.html](https://docs.python.org/3.8/library/time.html)
- [https://docs.python.org/3.8/library/timeit.html](https://docs.python.org/3.8/library/timeit.html)
- [https://docs.python.org/3.8/library/profile.html](https://docs.python.org/3.8/library/profile.html)

#### Last Updated: 18-Sep-2019